In [2]:
# imports
from jinja2 import Environment, FileSystemLoader
from pprint import pprint
import re

In [3]:
from re import Pattern

class TbData():
    dados:list = []
    texto:str = ""
    def __init__(self, texto:str):
        #converter string para dicionario
        self.dados = self.converterStringToDicionario(texto)
        self.texto = texto

    def converterStringToDicionario(self, texto: str) -> list:
        # Define um padrão regex para identificar objetos com seus nomes e tipos
        object_pattern: Pattern = re.compile(r"object (\w+): (\w+).*?>.*?end", re.DOTALL)
        
        # Define um padrão regex para identificar a seção FieldDefs e seu conteúdo
        fielddefs_pattern: Pattern = re.compile(r"FieldDefs\s*=\s*<\s*(.*?)\s*>\n\s*", re.DOTALL)
        
        # Define um padrão regex para identificar itens dentro da seção FieldDefs
        item_pattern: Pattern = re.compile(r"item\s+(.*?)\send", re.DOTALL)
        
        # Define um padrão regex para identificar pares chave-valor em cada item
        attr_pattern: Pattern = re.compile(r"(\w+) = '*(.*?)'*$", re.MULTILINE)
        
        objects = []  # Lista que armazenará todos os objetos convertidos em dicionários
        
        # Itera sobre todos os objetos encontrados no texto usando o padrão de objeto
        for obj_match in object_pattern.finditer(texto):
            # Extrai o nome e tipo do objeto da correspondência
            obj_name = obj_match.group(1)
            obj_type = obj_match.group(2)
            
            # Extrai o texto completo do objeto correspondente
            obj_texto = obj_match.group(0)
            
            # Cria um dicionário para o objeto atual, inicializando a chave 'FieldDefs' com uma lista vazia
            current_object = {'name': obj_name, 'type': obj_type, 'FieldDefs': []}
            
            # Itera sobre todos os itens encontrados no texto do objeto usando o padrão de item
            for item_match in item_pattern.finditer(obj_texto):
                current_item = {}  # Dicionário que armazenará os pares chave-valor para o item atual
                item_text = item_match.group(1)  # Extrai o texto do item
                
                # Extrai pares chave-valor do texto do item usando o padrão de atributo
                for attr_match in attr_pattern.finditer(item_text):
                    key, value = attr_match.group(1), attr_match.group(2)
                    current_item[key] = value  # Adiciona o par chave-valor ao dicionário do item

                # Adiciona o item processado à lista 'FieldDefs' do objeto atual
                current_object['FieldDefs'].append(current_item)
            

            # ordena a lista de objetos 'FieldDefs' de acordo com a linha e coluna
            current_object['FieldDefs'] = sorted(current_object['FieldDefs'], key=lambda x: (int(x.get('lin', 9999)), int(x.get('col', 9999))))

            # Remove a seção FieldDefs do texto do objeto para processar atributos restantes
            remaining_attrs = re.sub(fielddefs_pattern, '', obj_texto)
            
            # Itera sobre os pares chave-valor restantes no texto do objeto
            for attr_object_match in attr_pattern.finditer(remaining_attrs):
                key, value = attr_object_match.group(1), attr_object_match.group(2)
                current_object[key] = value  # Adiciona os pares chave-valor restantes ao dicionário do objeto
            
            # Adiciona o dicionário do objeto à lista de objetos
            objects.append(current_object)
        
        # Retorna a lista de objetos convertidos em dicionários
        return objects


In [4]:
from io import TextIOWrapper
#abre o arquivo
arquivoTabela:TextIOWrapper = open('tbPergunta.txt', 'r')
textoTabela = arquivoTabela.read()
arquivoTabela.close()

# inicializando variáveis da tabela
tbData:TbData = TbData(textoTabela)

In [5]:
pprint(tbData.dados)

[{'Cursor': 'CRM_PERGUNTAS',
  'DeltaMode': 'dmChanged',
  'FieldDefs': [{'Calculated': 'False',
                 'Caption': "C'#243'd. Pergunta",
                 'FieldType': 'ftInteger',
                 'Name': 'COD_PERGUNTA',
                 'NullOnEmpty': 'False',
                 'PrimaryKey': 'True',
                 'Updatable': 'False',
                 'WOrigem': 'EhNone',
                 'WOwner': 'FrWizard',
                 'col': '1',
                 'lin': '1'},
                {'Calculated': 'False',
                 'Caption': "C'#243'd. Questionario",
                 'FieldType': 'ftInteger',
                 'Name': 'COD_QUESTIONARIO',
                 'NullOnEmpty': 'False',
                 'PrimaryKey': 'True',
                 'Updatable': 'False',
                 'WOrigem': 'EhNone',
                 'WOwner': 'FrWizard',
                 'col': '1',
                 'lin': '2'}],
  'MaxRowCount': '200',
  'ModelType': 'mtCollectionModel',
  'OnAfterScroll

In [9]:
# Configura o carregador de templates para a pasta do script
env = Environment(loader=FileSystemLoader('templates'))
env.trim_blocks = True
env.lstrip_blocks = True

#renderizando o template
template = env.get_template('template_init.jinja')
print(template.render(data = tbData.dados))

object groupboxtbPerguntas: TFGroupbox
  Width = 745
  Height = 450
  Caption = 'Detalhes tbPerguntas'
  Font.Charset = DEFAULT_CHARSET
  Font.Color = clWindowText
  Font.Height = -11
  Font.Name = 'Tahoma'
  Font.Style = []
  Padding.Top = 0
  Padding.Left = 0
  Padding.Right = 0
  Padding.Bottom = 0
  ParentFont = False
  TabOrder = 0
  Margin.Top = 0
  Margin.Left = 0
  Margin.Right = 0
  Margin.Bottom = 0
  Flex.Vflex = ftTrue
  Flex.Hflex = ftTrue
  WOwner = FrInterno
  WOrigem = EhNone
  Scrollable = False
  Closable = False
  Closed = False
  Orient = coHorizontal
  Style = grp3D
  HeaderImageId = 0
  object vboxDetalhetbPerguntas: TFVBox
    Left = 2
    Top = 15
    Width = 745
    Height = 450
    Align = alClient
    AutoWrap = False
    BevelKind = bkTile
    BevelOuter = bvNone
    BorderStyle = stNone
    Caption = ''
    FlowStyle = fsTopBottomLeftRight
    Padding.Top = 5
    Padding.Left = 5
    Padding.Right = 10
    Padding.Bottom = 5
    TabOrder = 0
    Margin.Top 

In [8]:
import re

def converterStringToDicionario(texto: str) -> list:
    object_pattern:Pattern = re.compile(r"object (\w+): (\w+).*?>.*?end", re.DOTALL)
    fielddefs_pattern:Pattern = re.compile(r"FieldDefs\s*=\s*<\s*(.*?)\s*>\n\s*", re.DOTALL)
    item_pattern:Pattern = re.compile(r"item\s+(.*?)\send", re.DOTALL)
    attr_pattern:Pattern = re.compile(r"(\w+) = (.*?)$", re.MULTILINE)

    objects = []

    # Iterate over all objects in the text
    for obj_match in object_pattern.finditer(texto):
        obj_name = obj_match.group(1)
        obj_type = obj_match.group(2)
        obj_texto = obj_match.group(0)
        current_object = {'name': obj_name, 'type': obj_type, 'FieldDefs': []}
        for item_match in item_pattern.finditer(obj_texto):
                current_item = {}
                item_text = item_match.group(1)
                # Extract key-value pairs from each item
                for attr_match in attr_pattern.finditer(item_text):
                    key, value = attr_match.group(1), attr_match.group(2)
                    current_item[key] = value

                current_object['FieldDefs'].append(current_item)

        # Add other attributes of the object outside FieldDefs
        remaining_attrs =  re.sub(fielddefs_pattern, '', obj_texto) #pega o texto e remove o atibutos dos itens 
        for attr_object_match in attr_pattern.finditer(remaining_attrs):
          key, value = attr_object_match.group(1), attr_object_match.group(2)
          current_object[key] = value

        objects.append(current_object)

    return objects

tbtext = """
object tbPerguntas: TFTable
    FieldDefs = <
      item
        Name = 'COD_QUESTIONARIO'
        Calculated = False
        Updatable = False
        PrimaryKey = True
        FieldType = ftInteger
        JSONConfig.NullOnEmpty = False
        Caption = 'C'#243'd. Questionario'
        WOwner = FrWizard
        WOrigem = EhNone
      end
      item
        Name = 'COD_PERGUNTA'
        Calculated = False
        Updatable = False
        PrimaryKey = True
        FieldType = ftInteger
        JSONConfig.NullOnEmpty = False
        Caption = 'C'#243'd. Pergunta'
        WOwner = FrWizard
        WOrigem = EhNone
      end>
    TableName = 'CRM_PERGUNTAS'
    Cursor = 'CRM_PERGUNTAS'
    MaxRowCount = 200
    OnAfterScroll = tbPerguntasAfterScroll
    WOwner = FrWizard
    WOrigem = EhNone
    WKey = '43904;43902'
    DeltaMode = dmChanged
    RatioBatchSize = 20
    ModelType = mtCollectionModel
  end
object tbPerguntas: TFTable
    FieldDefs = <
      item
        Name = 'COD_QUESTIONARIO'
        Calculated = False
        Updatable = False
        PrimaryKey = True
        FieldType = ftInteger
        JSONConfig.NullOnEmpty = False
        Caption = 'C'#243'd. Questionario'
        WOwner = FrWizard
        WOrigem = EhNone
      end
      item
        Name = 'COD_PERGUNTA'
        Calculated = False
        Updatable = False
        PrimaryKey = True
        FieldType = ftInteger
        JSONConfig.NullOnEmpty = False
        Caption = 'C'#243'd. Pergunta'
        WOwner = FrWizard
        WOrigem = EhNone
      end>
    TableName = 'CRM_PERGUNTAS'
    Cursor = 'CRM_PERGUNTAS'
    MaxRowCount = 200
    OnAfterScroll = tbPerguntasAfterScroll
    WOwner = FrWizard
    WOrigem = EhNone
    WKey = '43904;43902'
    DeltaMode = dmChanged
    RatioBatchSize = 20
    ModelType = mtCollectionModel
  end
"""
import pprint
teste = converterStringToDicionario(tbtext)
pprint.pprint(teste)

[{'Cursor': "'CRM_PERGUNTAS'",
  'DeltaMode': 'dmChanged',
  'FieldDefs': [{'Calculated': 'False',
                 'Caption': "'C'#243'd. Questionario'",
                 'FieldType': 'ftInteger',
                 'Name': "'COD_QUESTIONARIO'",
                 'NullOnEmpty': 'False',
                 'PrimaryKey': 'True',
                 'Updatable': 'False',
                 'WOrigem': 'EhNone',
                 'WOwner': 'FrWizard'},
                {'Calculated': 'False',
                 'Caption': "'C'#243'd. Pergunta'",
                 'FieldType': 'ftInteger',
                 'Name': "'COD_PERGUNTA'",
                 'NullOnEmpty': 'False',
                 'PrimaryKey': 'True',
                 'Updatable': 'False',
                 'WOrigem': 'EhNone',
                 'WOwner': 'FrWizard'}],
  'MaxRowCount': '200',
  'ModelType': 'mtCollectionModel',
  'OnAfterScroll': 'tbPerguntasAfterScroll',
  'RatioBatchSize': '20',
  'TableName': "'CRM_PERGUNTAS'",
  'WKey': "'43904